# IAPI Bridge Test

Stand-alone codes using Pythonnet to connect to IAPI

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os
import glob
import datetime
import time
from pathlib import Path

from IPython import display
import pylab as plt
import numpy as np
import clr
import atexit

## Calling the IAPI Bridge from Python

### Load our custom assembly

In [3]:
ref_dir = 'C:\\Users\\joewa\\Work\\git\\vimms\\Interface\\FusionLibrary\\bin\\Debug\\netstandard2.0'
if ref_dir not in sys.path:
    sys.path.append(ref_dir)

In [4]:
print(clr.FindAssembly('FusionLibrary'))

C:\Users\joewa\Work\git\vimms\Interface\FusionLibrary\bin\Debug\netstandard2.0\FusionLibrary.dll


In [5]:
list(clr.ListAssemblies(False))

['mscorlib',
 'clrmodule',
 'Python.Runtime',
 'System.Core',
 'System.Configuration',
 'System.Xml',
 'System',
 '__CodeGenerator_Assembly',
 'e__NativeCall_Assembly']

In [6]:
ref = clr.AddReference('FusionLibrary')

In [7]:
list(clr.ListAssemblies(False))

['mscorlib',
 'clrmodule',
 'Python.Runtime',
 'System.Core',
 'System.Configuration',
 'System.Xml',
 'System',
 '__CodeGenerator_Assembly',
 'e__NativeCall_Assembly',
 'FusionLibrary',
 'netstandard',
 'Fusion.API-1.0',
 'API-2.0',
 'Spectrum-1.0']

### Create FusionBridge

Below codes generate a bunch of custom scans

In [8]:
from FusionLibrary import FusionBridge

In [9]:
filename = 'C:\\Users\\joewa\\University of Glasgow\\Vinny Davies - CLDS Metabolomics Project\\Data\\multibeers_urine_data\\beers\\fragmentation\\mzML\\Beer_multibeers_1_T10_POS.mzML'
# filename = None # to connect to the actual machine

In [10]:
def scan_handler(scan):
    print('Received scan %s -- %d peaks' % (scan.Header['Scan'], scan.CentroidCount))
    
def state_changed_handler(state):
    print('state_change_handler is called')

class CustomScanHandler(object):
    def __init__(self, fusion_bridge, start_mz, end_mz, isolation_width, collision_energy):
        self.start_mz = start_mz
        self.end_mz = end_mz
        self.isolation_width = isolation_width
        self.collision_energy = collision_energy
        self.precursor_mass = self.start_mz
        self.fusion_bridge = fusion_bridge
            
    def step(self):
        print('custom_scan_handler called')
        if self.precursor_mass < self.end_mz:
            self.precursor_mass += 0.02
            self.fusion_bridge.CreateCustomScan(self.precursor_mass, self.isolation_width, self.collision_energy, 2)

In [11]:
start_mz = 524.0
end_mz = 524.5
isolation_width = 0.7
collision_energy = 35

In [12]:
fusion_bridge = FusionBridge(filename)
atexit.register(fusion_bridge.CloseDown)

# create event handlers
scan_handler_delegate = FusionBridge.UserScanArriveDelegate(scan_handler)
state_changed_delegate = FusionBridge.UserStateChangedDelegate(state_changed_handler)
scan_handler = CustomScanHandler(fusion_bridge, start_mz, end_mz, isolation_width, collision_energy)
custom_scan_delegate = FusionBridge.UserCreateCustomScanDelegate(scan_handler.step)

fusion_bridge.SetEventHandlers(scan_handler_delegate, state_changed_delegate, custom_scan_delegate)
fusion_bridge.CreateCustomScan(start_mz, isolation_width, collision_energy, 2)

custom_scan_handler called
custom_scan_handler called
custom_scan_handler called
custom_scan_handler called
custom_scan_handler called
custom_scan_handler called
custom_scan_handler called
custom_scan_handler called
custom_scan_handler called
custom_scan_handler called
custom_scan_handler called
custom_scan_handler called
custom_scan_handler called
custom_scan_handler called
custom_scan_handler called
custom_scan_handler called
custom_scan_handler called
custom_scan_handler called
custom_scan_handler called
custom_scan_handler called
custom_scan_handler called
custom_scan_handler called
custom_scan_handler called
custom_scan_handler called
custom_scan_handler called
custom_scan_handler called
custom_scan_handler called
Received scan 2 -- 16 peaks
Received scan 3 -- 20 peaks
Received scan 4 -- 22 peaks
Received scan 5 -- 14 peaks
Received scan 6 -- 18 peaks
Received scan 7 -- 33 peaks
Received scan 8 -- 8 peaks
Received scan 9 -- 13 peaks
Received scan 10 -- 11 peaks
Received scan 11 --